In [216]:
import numpy as np
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from datetime import datetime, timedelta
import json

def load_google_sheets_data(worksheet_name: str):
    """Google Sheets에서 데이터 로드 - 중복 헤더 오류 해결"""
    
    with open('pj_appscript.json', 'r') as f:
        credentials_info = json.load(f)

    scope = [
        "https://spreadsheets.google.com/feeds",
        "https://www.googleapis.com/auth/drive"
    ]
    creds = ServiceAccountCredentials.from_json_keyfile_dict(credentials_info, scope)
    client = gspread.authorize(creds)

    # 스프레드시트 열기
    spreadsheet = client.open("🔥🔥🔥 경험그룹_KPI (수업 기준!!!!!) 🔥🔥🔥")
    worksheet = spreadsheet.worksheet(worksheet_name)

    try:
        # 모든 데이터 가져오기
        all_values = worksheet.get_all_values()
        
        if not all_values:
            print("워크시트가 비어있습니다.")
            return pd.DataFrame()
        
        # 두 번째 행을 헤더로 사용 (첫 번째 행은 스킵)
        headers = all_values[1]
        data_rows = all_values[2:]
        
        # 중복된 빈 헤더 처리
        processed_headers = []
        for i, header in enumerate(headers):
            if header == '' or header in processed_headers:
                # 빈 헤더나 중복 헤더에 고유한 이름 부여
                processed_headers.append(f'unnamed_column_{i}')
            else:
                processed_headers.append(header)
        
        # DataFrame 생성
        df = pd.DataFrame(data_rows, columns=processed_headers)
        
        # 빈 행 제거
        df = df.dropna(how='all')
        
        # 불필요한 unnamed 컬럼들 제거 (모든 값이 비어있는 경우)
        cols_to_drop = []
        for col in df.columns:
            if col.startswith('unnamed_column_') and (df[col].isna().all() or (df[col] == '').all()):
                cols_to_drop.append(col)
        
        df = df.drop(columns=cols_to_drop)
        
        print(f"데이터 로드 성공: {len(df)}행, {len(df.columns)}열")
        return df
        
    except Exception as e:
        print(f"데이터 로드 중 오류: {str(e)}")
        return pd.DataFrame()

def load_google_sheets_data_alternative(worksheet_name: str):
    """Google Sheets에서 데이터 로드 - 대안 방법 (범위 지정)"""
    
    with open('pj_appscript.json', 'r') as f:
        credentials_info = json.load(f)

    scope = [
        "https://spreadsheets.google.com/feeds",
        "https://www.googleapis.com/auth/drive"
    ]
    creds = ServiceAccountCredentials.from_json_keyfile_dict(credentials_info, scope)
    client = gspread.authorize(creds)

    # 스프레드시트 열기
    spreadsheet = client.open("🔥🔥🔥 경험그룹_KPI (수업 기준!!!!!) 🔥🔥🔥")
    worksheet = spreadsheet.worksheet(worksheet_name)

    try:
        # 첫 번째 행(헤더) 확인 - 두 번째 행을 헤더로 사용
        second_row = worksheet.row_values(2)
        
        # 실제 데이터가 있는 컬럼까지만 확인
        last_col_with_data = 0
        for i, val in enumerate(second_row):
            if val.strip():  # 공백이 아닌 값이 있으면
                last_col_with_data = i + 1
        
        if last_col_with_data == 0:
            print("두 번째 행(헤더)에 유효한 데이터가 없습니다.")
            return pd.DataFrame()
        
        # 유효한 범위의 데이터만 가져오기 (2행부터 시작)
        range_name = f'A2:{chr(65 + last_col_with_data - 1)}'  # A2:Z 형식
        data = worksheet.get(range_name)
        
        if len(data) < 2:
            print("데이터가 충분하지 않습니다.")
            return pd.DataFrame()
        
        headers = data[0][:last_col_with_data]  # 첫 번째 가져온 행이 헤더
        rows = [row[:last_col_with_data] for row in data[1:]]  # 나머지가 데이터
        
        # 행 길이 맞추기 (일부 행이 짧을 수 있음)
        max_cols = len(headers)
        normalized_rows = []
        for row in rows:
            normalized_row = row + [''] * (max_cols - len(row))
            normalized_rows.append(normalized_row[:max_cols])
        
        # DataFrame 생성
        df = pd.DataFrame(normalized_rows, columns=headers)
        
        print(f"데이터 로드 성공: {len(df)}행, {len(df.columns)}열")
        return df
        
    except Exception as e:
        print(f"대안 방법으로 데이터 로드 중 오류: {str(e)}")
        return pd.DataFrame()

def inspect_worksheet_structure(worksheet_name: str):
    """워크시트 구조 확인"""
    
    with open('pj_appscript.json', 'r') as f:
        credentials_info = json.load(f)

    scope = [
        "https://spreadsheets.google.com/feeds",
        "https://www.googleapis.com/auth/drive"
    ]
    creds = ServiceAccountCredentials.from_json_keyfile_dict(credentials_info, scope)
    client = gspread.authorize(creds)

    spreadsheet = client.open("🔥🔥🔥 경험그룹_KPI (수업 기준!!!!!) 🔥🔥🔥")
    worksheet = spreadsheet.worksheet(worksheet_name)
    
    # 두 번째 행 확인 (실제 헤더)
    second_row = worksheet.row_values(2)
    
    print("=== 워크시트 헤더 구조 분석 (2행 기준) ===")
    print(f"총 컬럼 수: {len(second_row)}")
    print()
    
    for i, header in enumerate(second_row):
        if header.strip():
            print(f"컬럼 {i+1}: '{header}'")
        else:
            print(f"컬럼 {i+1}: [빈 헤더]")
    
    # 빈 헤더나 중복 헤더 확인
    empty_headers = [i+1 for i, h in enumerate(second_row) if not h.strip()]
    duplicates = []
    seen = {}
    for i, h in enumerate(second_row):
        if h in seen and h.strip():  # 빈 문자열은 중복으로 간주하지 않음
            duplicates.append((i+1, h))
        else:
            seen[h] = i+1
    
    print("\n=== 문제점 분석 ===")
    if empty_headers:
        print(f"빈 헤더 위치: {empty_headers}")
    if duplicates:
        print(f"중복 헤더: {duplicates}")
    
    if not empty_headers and not duplicates:
        print("헤더에 문제없음")

# 사용 예시
if __name__ == "__main__":
    # 1. 먼저 워크시트 구조 확인
    print("1. 워크시트 구조 확인")
    inspect_worksheet_structure("이탈_RAW")
    
    print("\n" + "="*50 + "\n")
    
    # 2. 데이터 로드 시도 (방법 1)
    print("2. 방법 1: 데이터 로드 (중복 헤더 처리)")
    df = load_google_sheets_data("이탈_RAW")
    
    if not df.empty:
        print(f"행 수: {len(df)}, 열 수: {len(df.columns)}")
        print("컬럼명:", df.columns.tolist())
        print("\n첫 5행:")
        print(df.head())
    
    print("\n" + "="*50 + "\n")
    
    # 3. 대안 방법으로 시도
    print("3. 방법 2: 데이터 로드 (범위 지정)")
    df_alt = load_google_sheets_data_alternative("이탈_RAW")
    
    if not df_alt.empty:
        print(f"행 수: {len(df_alt)}, 열 수: {len(df_alt.columns)}")
        print("컬럼명:", df_alt.columns.tolist())
        print("\n첫 5행:")
        print(df_alt.head())

# 간단한 사용법:
# df = load_google_sheets_data("이탈_RAW")

1. 워크시트 구조 확인
=== 워크시트 헤더 구조 분석 (2행 기준) ===
총 컬럼 수: 35

컬럼 1: '단계'
컬럼 2: 'payment_regdate'
컬럼 3: '이탈여부'
컬럼 4: 'user_No'
컬럼 5: 'lvt'
컬럼 6: '과외상태'
컬럼 7: 'stage_count'
컬럼 8: 'cycle_count'
컬럼 9: '수업상태'
컬럼 10: '과외일정'
컬럼 11: '과목'
컬럼 12: '학생이름'
컬럼 13: 'application_datetime'
컬럼 14: 'fst_matchedat'
컬럼 15: '튜터이름'
컬럼 16: '튜터유저넘버'
컬럼 17: [빈 헤더]
컬럼 18: [빈 헤더]
컬럼 19: [빈 헤더]
컬럼 20: [빈 헤더]
컬럼 21: 'reactive_datetime'
컬럼 22: 'submitdat'
컬럼 23: 'submit_status'
컬럼 24: 'reason'
컬럼 25: 'reason_detail'
컬럼 26: 'email_id'
컬럼 27: 'done_month'
컬럼 28: '교과/탐구'
컬럼 29: '학년'
컬럼 30: 'option'
컬럼 31: '중단예정일'
컬럼 32: '중단 예정 DONEMONTH'
컬럼 33: '최초 개월 수'
컬럼 34: 'done_count'
컬럼 35: 'glvt_no'

=== 문제점 분석 ===
빈 헤더 위치: [17, 18, 19, 20]


2. 방법 1: 데이터 로드 (중복 헤더 처리)
데이터 로드 성공: 37728행, 35열
행 수: 37728, 열 수: 35
컬럼명: ['단계', 'payment_regdate', '이탈여부', 'user_No', 'lvt', '과외상태', 'stage_count', 'cycle_count', '수업상태', '과외일정', '과목', '학생이름', 'application_datetime', 'fst_matchedat', '튜터이름', '튜터유저넘버', 'unnamed_column_16', 'unnamed_column_17', 

In [217]:
df = load_google_sheets_data("이탈_RAW")

데이터 로드 성공: 37728행, 35열


In [228]:
# 절대 필수 컬럼 (생존분석 핵심)
essential_columns = [
    'payment_regdate',  # 결제일 (생존분석 시작점)
    '중단예정일',        # 이탈 시점 (P인 경우)
    '이탈여부',         # P(이탈), A(활성), T(테스트)
    '단계',            # 현재 진행 단계 (1~9)
    'user_No',         # 학생 식별자
    'lvt'              # 수업 식별자
]

# 분석 품질 향상용 컬럼 (모두 사용 가능)
recommended_columns = [
    'done_month',           # 현재 누적 수업 기간
    '중단 예정 DONEMONTH',   # 이탈 시점의 수업 기간 (공백 주의!)
    '최초 개월 수',         # 계약 개월수 (공백 주의!)
    '교과/탐구',           # 과목 분류
    '학년',               # 학생 학년
    'option',             # 수업 타입 (W1_H90, W2_H60 등)
    '과목'                # 세부 과목
]

# 전체 사용 가능한 컬럼
all_available_columns = essential_columns + recommended_columns

print("사용 가능한 모든 컬럼:", all_available_columns)

사용 가능한 모든 컬럼: ['payment_regdate', '중단예정일', '이탈여부', '단계', 'user_No', 'lvt', 'done_month', '중단 예정 DONEMONTH', '최초 개월 수', '교과/탐구', '학년', 'option', '과목']


In [224]:
df.head()

,단계,payment_regdate,이탈여부,user_No,lvt,과외상태,stage_count,cycle_count,수업상태,과외일정,...,email_id,done_month,교과/탐구,학년,option,중단예정일,중단 예정 DONEMONTH,최초 개월 수,done_count,glvt_no
0,9,2024-01-01 10:37,P,612220,72405,FINISH,10,8,RESERVATION,2025-03-03 21:00:00,...,daeun01@naver.com,15.125,교과,중3,W2_H60,2025-02-28 12:10,15.125,6,121,
1,9,2024-01-01 13:17,P,620252,78506,FINISH,4,2,RESERVATION,2024-11-15 22:30:00,...,won95580@naver.com,9.7493,교과,N수생,W2_H60,2024-07-08 10:11,9.7493,6,85,
2,9,2024-01-01 13:17,P,620252,78508,FINISH,4,8,RESERVATION,2024-09-01 13:00:00,...,won95580@naver.com,6.0396,교과,N수생,W2_H90,2024-10-07 10:59,6.0396,3,70,
3,8,2024-01-01 13:38,P,620153,78462,FINISH,1,16,RESERVATION,2024-05-25 10:00:00,...,victoryzoo1231@naver.com,3.75,교과,고3,W1_H90,2024-05-23 14:46,3.75,6,15,
4,9,2024-01-01 14:07,P,608191,78509,AUTO_FINISH,15,4,DONE,2025-04-29 20:30:00,...,psw492@gmail.com,17.5,교과,중2,W1_H60,2025-04-28 13:31,17.75,3,71,


In [226]:
df = df[essential_columns+recommended_columns]
df.head()

,payment_regdate,중단예정일,이탈여부,단계,user_No,lvt,done_month,중단 예정 DONEMONTH,최초 개월 수,교과/탐구,학년,option,과목
0,2024-01-01 10:37,2025-02-28 12:10,P,9,612220,72405,15.125,15.125,6,교과,중3,W2_H60,수학
1,2024-01-01 13:17,2024-07-08 10:11,P,9,620252,78506,9.7493,9.7493,6,교과,N수생,W2_H60,수학
2,2024-01-01 13:17,2024-10-07 10:59,P,9,620252,78508,6.0396,6.0396,3,교과,N수생,W2_H90,영어
3,2024-01-01 13:38,2024-05-23 14:46,P,8,620153,78462,3.75,3.75,6,교과,고3,W1_H90,영어
4,2024-01-01 14:07,2025-04-28 13:31,P,9,608191,78509,17.5,17.75,3,교과,중2,W1_H60,수학


In [227]:
df.columns

Index(['payment_regdate', '중단예정일', '이탈여부', '단계', 'user_No', 'lvt',
       'done_month', '중단 예정 DONEMONTH', '최초 개월 수', '교과/탐구', '학년', 'option',
       '과목'],
      dtype='object')

In [231]:
import pandas as pd
from datetime import datetime

def process_survival_data(df):
    """생존분석용 데이터 처리"""
    
    print("=== 데이터 처리 시작 ===")
    original_len = len(df)
    
    # 1. 테스트 데이터 제외
    df_clean = df[df['이탈여부'] != 'T'].copy()
    print(f"테스트 제외: {original_len} → {len(df_clean)} ({original_len - len(df_clean)} 제거)")
    
    # 2. 필수 컬럼 빈값 제거
    essential_cols = ['payment_regdate', '중단예정일', '이탈여부', '단계']
    for col in essential_cols:
        before = len(df_clean)
        df_clean = df_clean[df_clean[col].notna()]
        after = len(df_clean)
        if before != after:
            print(f"{col} 빈값 제거: {before} → {after} ({before-after} 제거)")
    
    # 3. 날짜 컬럼 변환 (에러 처리 포함)
    print("날짜 컬럼 변환 중...")
    
    # payment_regdate 변환
    try:
        df_clean['payment_regdate'] = pd.to_datetime(df_clean['payment_regdate'])
        print("✅ payment_regdate 변환 완료")
    except Exception as e:
        print(f"❌ payment_regdate 변환 실패: {e}")
        # 다양한 형식 시도
        df_clean['payment_regdate'] = pd.to_datetime(df_clean['payment_regdate'], errors='coerce')
    
    # 중단예정일 변환 (더 유연하게)
    try:
        # 먼저 어떤 형식의 데이터가 있는지 확인
        print(f"중단예정일 샘플 데이터:")
        sample_dates = df_clean['중단예정일'].dropna().head(10).tolist()
        for i, date in enumerate(sample_dates):
            print(f"  {i+1}. '{date}'")
        
        # 다양한 형식으로 시도
        df_clean['중단예정일'] = pd.to_datetime(df_clean['중단예정일'], format='mixed', errors='coerce')
        print("✅ 중단예정일 변환 완료 (mixed format)")
        
    except Exception as e:
        print(f"❌ 중단예정일 변환 실패: {e}")
        print("대안 방법 시도...")
        
        # 대안: 수동으로 정리
        def clean_date_string(date_str):
            if pd.isna(date_str) or date_str == '':
                return None
            
            # 문자열로 변환
            date_str = str(date_str).strip()
            
            # 일반적인 문제 패턴 정리
            if ':17' in date_str:  # 에러 메시지에서 본 패턴
                date_str = date_str.replace(':17', '')
            
            # 기타 정리 작업
            date_str = date_str.replace('  ', ' ')  # 이중 공백 제거
            
            return date_str
        
        # 데이터 정리 후 재시도
        df_clean['중단예정일_cleaned'] = df_clean['중단예정일'].apply(clean_date_string)
        df_clean['중단예정일'] = pd.to_datetime(df_clean['중단예정일_cleaned'], errors='coerce')
        df_clean = df_clean.drop('중단예정일_cleaned', axis=1)
        print("✅ 중단예정일 정리 후 변환 완료")
    
    # 변환 결과 확인
    payment_na = df_clean['payment_regdate'].isna().sum()
    end_na = df_clean['중단예정일'].isna().sum()
    
    print(f"날짜 변환 결과:")
    print(f"  payment_regdate 빈값: {payment_na}개")
    print(f"  중단예정일 빈값: {end_na}개")
    
    # 날짜 변환 실패한 행 제거
    if payment_na > 0:
        df_clean = df_clean[df_clean['payment_regdate'].notna()]
        print(f"payment_regdate 빈값 제거: {payment_na}개")
    
    # 이탈한 경우(P)인데 중단예정일이 없는 경우 제거
    p_without_end_date = df_clean[(df_clean['이탈여부'] == 'P') & (df_clean['중단예정일'].isna())]
    if len(p_without_end_date) > 0:
        print(f"⚠️ 이탈(P)인데 중단예정일 없음: {len(p_without_end_date)}개 → 제거")
        df_clean = df_clean[~((df_clean['이탈여부'] == 'P') & (df_clean['중단예정일'].isna()))]
    
    # 4. 분석 기간 필터링 (2023-05-01 ~ 2024-04-30)
    analysis_start = pd.to_datetime('2023-05-01')
    analysis_end = pd.to_datetime('2024-04-30')
    
    before_period = len(df_clean)
    df_clean = df_clean[
        (df_clean['payment_regdate'] >= analysis_start) & 
        (df_clean['payment_regdate'] <= analysis_end)
    ]
    print(f"분석 기간 필터링: {before_period} → {len(df_clean)} ({before_period - len(df_clean)} 제거)")
    
    # 5. 생존분석 변수 생성
    # 관찰 종료일 계산
    df_clean['관찰종료일'] = df_clean.apply(lambda row: 
        row['중단예정일'] if row['이탈여부'] == 'P' 
        else pd.Timestamp.now(), axis=1)
    
    # 생존시간 계산 (일 단위)
    df_clean['생존일수'] = (df_clean['관찰종료일'] - df_clean['payment_regdate']).dt.days
    
    # 생존시간 (월 단위)
    df_clean['생존개월수'] = df_clean['생존일수'] / 30.0
    
    # 이벤트 발생 여부 (0/1)
    df_clean['이벤트발생'] = (df_clean['이탈여부'] == 'P').astype(int)
    
    # 6. 결과 요약
    print(f"\n=== 처리 완료 ===")
    print(f"최종 데이터: {len(df_clean):,}개 행")
    print(f"이탈률: {df_clean['이벤트발생'].mean():.1%}")
    print(f"평균 생존기간: {df_clean['생존개월수'].mean():.1f}개월")
    print(f"중위 생존기간: {df_clean['생존개월수'].median():.1f}개월")
    
    # 단계별 분포
    print(f"\n단계별 분포:")
    stage_dist = df_clean['단계'].value_counts().sort_index()
    for stage, count in stage_dist.items():
        pct = count / len(df_clean) * 100
        print(f"  단계 {stage}: {count:,}개 ({pct:.1f}%)")
    
    return df_clean

def validate_data_quality(df):
    """데이터 품질 검증"""
    
    print("=== 데이터 품질 검증 ===")
    
    # 1. 시간 순서 검증
    invalid_times = df[df['생존일수'] < 0]
    if len(invalid_times) > 0:
        print(f"⚠️ 음수 생존시간: {len(invalid_times)}개")
    else:
        print("✅ 시간 순서 정상")
    
    # 2. 생존시간 분포 확인
    print(f"생존시간 분포:")
    print(f"  최소: {df['생존일수'].min()}일")
    print(f"  최대: {df['생존일수'].max()}일")
    print(f"  평균: {df['생존일수'].mean():.0f}일")
    
    # 3. 이상치 확인
    outliers = df[df['생존일수'] > 365*2]  # 2년 초과
    if len(outliers) > 0:
        print(f"⚠️ 2년 초과 데이터: {len(outliers)}개")
    else:
        print("✅ 이상치 없음")
    
    return len(invalid_times) == 0

# 전체 처리 파이프라인
def full_pipeline(df):
    """전체 데이터 처리 파이프라인"""
    
    print("🚀 AUC 생존분석 데이터 처리 시작")
    print("=" * 50)
    
    # 1. 데이터 처리
    df_processed = process_survival_data(df)
    
    # 2. 품질 검증
    is_valid = validate_data_quality(df_processed)
    
    if is_valid:
        print("\n✅ 데이터 처리 완료! 생존분석 준비됨")
        return df_processed
    else:
        print("\n⚠️ 데이터 품질 문제 발견. 추가 정제 필요")
        return df_processed

In [232]:
df = load_google_sheets_data("이탈_RAW")

# 전체 처리
df_final = full_pipeline(df)

# 바로 생존분석 시작 가능!
print("생존분석 준비 완료!")

데이터 로드 성공: 37728행, 35열
🚀 AUC 생존분석 데이터 처리 시작
=== 데이터 처리 시작 ===
테스트 제외: 37728 → 37718 (10 제거)
날짜 컬럼 변환 중...
✅ payment_regdate 변환 완료
중단예정일 샘플 데이터:
  1. '2025-02-28 12:10'
  2. '2024-07-08 10:11'
  3. '2024-10-07 10:59'
  4. '2024-05-23 14:46'
  5. '2025-04-28 13:31'
  6. '2024-03-25 18:30:17'
  7. ''
  8. '2025-03-08 14:15'
  9. '2024-09-19 16:55:52'
  10. '2024-12-14 14:33:12'
✅ 중단예정일 변환 완료 (mixed format)
날짜 변환 결과:
  payment_regdate 빈값: 0개
  중단예정일 빈값: 11648개
⚠️ 이탈(P)인데 중단예정일 없음: 1853개 → 제거
분석 기간 필터링: 35865 → 6314 (29551 제거)

=== 처리 완료 ===
최종 데이터: 6,314개 행
이탈률: 91.7%
평균 생존기간: 6.7개월
중위 생존기간: 4.9개월

단계별 분포:
  단계 1: 65개 (1.0%)
  단계 2: 34개 (0.5%)
  단계 3: 190개 (3.0%)
  단계 4: 381개 (6.0%)
  단계 5: 92개 (1.5%)
  단계 6: 9개 (0.1%)
  단계 7: 110개 (1.7%)
  단계 8: 931개 (14.7%)
  단계 9: 4,502개 (71.3%)
=== 데이터 품질 검증 ===
⚠️ 음수 생존시간: 1개
생존시간 분포:
  최소: -1011일
  최대: 636일
  평균: 200일
✅ 이상치 없음

⚠️ 데이터 품질 문제 발견. 추가 정제 필요
생존분석 준비 완료!


In [ ]:
# 품질 문제 자동 수정
df_fixed = fix_data_quality_issues(df_final)

# 재검증
is_valid_after_fix = validate_data_quality(df_fixed)

print(f"수정 후 데이터: {len(df_fixed)}개")